In [1]:
# Import Statements
import cv2 as cv
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from matplotlib import pyplot as plt
import time
import sys
import multiprocessing
import os
import rospy
from sensor_msgs.msg import Image
from stereo_msgs.msg import DisparityImage
from sensor_msgs.msg import PointCloud2
from nav_msgs.msg import OccupancyGrid as Map
import message_filters
import cv_bridge
import tf
import math
from std_msgs.msg import String
import EnvMap
import Buoy_List
from numba import jit

In [2]:
# Initialization Stuff

rospy.init_node('liveproc')

listener = tf.TransformListener()
pub = rospy.Publisher('updated_map', Map)


occ_map = None
up_occ_map = None
map_resolution = 0.5
origin = None
robot_x = 0
robot_y = 0
map_width = 0
b_list = []

map_height = 0
camera_range = 85/map_resolution
overlay = True
display_robot_yaw = False
display_rect = False
display_map = False
display_info = False
tri_width = 5
dist_error = 0.15

np.set_printoptions(threshold=sys.maxsize)

/home/alice/.local/lib/python2.7/site-packages/ipykernel_launcher.py:7: SyntaxWarning: The publisher should be created with an explicit keyword argument 'queue_size'. Please see http://wiki.ros.org/rospy/Overview/Publishers%20and%20Subscribers for more information.
  import sys


In [3]:
def disp_rect(rect):
    cv.namedWindow("stereo/left/image_rect_color",cv.WINDOW_NORMAL)
    cv.imshow("stereo/left/image_rect_color", rect)
    cv.waitKey(1)

In [4]:
def disp_map(map_img):
    map_img = np.swapaxes(map_img, 0, 1)
    cv.namedWindow("map_inverted",cv.WINDOW_NORMAL)
    cv.imshow("map_inverted", map_img)
    cv.waitKey(1)

In [5]:
def transform(img):
    start_time = time.time()
    global bearings
    global areas
    global tags    
    
    bridge = cv_bridge.CvBridge()
    
    areas = []
    bearings = []
    tags = []
    
    width = img.height
    height = img.width
    
    img = bridge.imgmsg_to_cv2(img, desired_encoding="passthrough")
    
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)  
    
    
    image = pp(img,width,height)
    
    img2= dbscan(image, img)
    print time.time()-start_time, "transform"
    return img2


def pp(image, width, height):
    start_time = time.time()
    image = np.array(image[0:width][0:int(height*0.65)])
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_and(image[:,:,1] > 5,np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10))))] = [0,0,0]
    image = cv.cvtColor(image,cv.COLOR_HSV2BGR)
    print time.time()-start_time, "preprocessing"
    return image

def dbscan(image, original):
    start_time = time.time()
    img = image.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret,thresh1 = cv.threshold(img,10,255,cv.THRESH_BINARY)

    nonzero = np.nonzero(thresh1)

    yp = np.array(nonzero[0])
    xp = np.array(nonzero[1])


    X=np.column_stack((xp,yp))

    if len(X) > 0:
        db = DBSCAN(eps=3, min_samples=10).fit(X)
        labels = db.labels_

        array = zip(xp,yp,labels)
        sort = sorted(list(array), key=lambda x: x[2])

        x = 0
        while len(sort) > 0 and sort[x][2] == -1:
            del sort[x]

        if len(sort) > 0:
            unique_labels = sorted(set(labels))
            if unique_labels[0] == -1:
                unique_labels.remove(-1)
            points = sort
            points = np.delete(points,2,1)

            i = 0
            x = 0
            l = []
            tb = None
            while x < len(unique_labels):
                if i < len(sort) and sort[i][2] == x:
                    l.append(points[i])
                    i+=1
                else:
                    x+=1
                    clr = color_hsv(np.array(l),image)
                    tb = rectangle(np.array(l),clr,image,original)
                    l = []
            original = tb
    print time.time()-start_time, "dbscan"
    return original

def color_hsv(mat, image):
    start_time = time.time()
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    sum_hue = 0
    sum_sat = 0
    sum_val = 0
    mat_add = 0
    for x in range(len(mat)):
        if image [mat[x][1]] [mat[x][0]] [2] > 5:
            mat_add = mat_add+1
            sum_val += image [mat[x][1]] [mat[x][0]] [2]
            sum_sat += image [mat[x][1]] [mat[x][0]] [1]
            sum_hue += image [mat[x][1]] [mat[x][0]] [0]
    if mat_add > 0:
        sat = sum_sat/mat_add
        val = sum_val/mat_add
        hue = sum_hue/mat_add
        #return (hue, sat, val)
        print time.time()-start_time, "color hsv"
        if sat > 5 and val > 5 and (hue < 60 or hue >= 170):
            return 'red'
        if sat > 5 and val > 5 and (70 < hue < 90):
            return 'green'
        else:
            return hue, sat, val
    
    return None

In [6]:
def rectangle(mat,color,image,original):
    start_time = time.time()
    global bearings
    global overlay
    global occ_map
    global tags
    global display_info
    global map_resolution
    global areas
    
    rect = cv.boundingRect(mat)
    min_rect = cv.minAreaRect(mat)

    height = rect[3]
    width = rect[2]
    x = rect[0]
    x = np.int0(x)
    y = rect[1] 
    y = np.int0(y)
    font = cv.FONT_HERSHEY_SIMPLEX
    tag = 0
    if overlay:
        if width < height < 4*width and width*height > 45:
            if color =='green':
                tag = 2
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,128,0),2,cv.LINE_AA)
                tags.append(tag)
            elif color =='red':
                tag = 3
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,0,128),2,cv.LINE_AA)
                tags.append(tag)
            else:
                tag = 1
                cv.putText(original,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            cv.rectangle(original,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = (min_rect[0][0]-640.0)/1280.0*80.0
            cv.putText(original,'bearing: '+ str(round(bearing,2)),(np.int0(x+width/2),np.int0(y+height/2.5)), font, 0.5,(0,0,0),2,cv.LINE_AA)
            bearings.append(bearing)
            area = min_rect[1][0]*min_rect[1][1]
            areas.append(area)
#             print("Area: " + str(area) + " and Tag: " + str(tag) +
#                   "\nEstimated Distance: " + str(2*map_resolution*456.7946397925*math.pow(area,-0.4705300238)))
            if display_info and occ_map.header.seq % 5 == 0:
                print bearings
                print tags
        if tag == 0:
            image[y:y+height][x:x+width]=[0,0,0]
        return original
    else:
        if width < height < 4*width and width*height > 45:
            if color =='green':
                tag = 2
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            elif color =='red':
                tag = 3
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            else:
                tag = 1
                cv.putText(image,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            cv.rectangle(image,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = (min_rect[0][0]-640.0)/1280.0*80.0
            bearings.append(bearing)
            area = min_rect[1][0]*min_rect[1][1]
            areas.append(area)
            
            if display_info and occ_map.header.seq % 5 == 0:
                print bearings
                print tags
        print time.time()-start_time, "rectangle"
        return image

In [7]:
def draw_background_objects(map_img, map_np):
    start_time = time.time()
    global origin
    global robot_x
    global robot_y
    
    o_x = (-origin.x)/map_resolution
    o_y = (-origin.y)/map_resolution
# Color Assignment Section
    map_img[map_np == -1] = 0    #Change to [255,255,255] for color to [0] for grayscale
    map_img[map_np == 0] = 0     #Change to [255,255,255] for color to [0] for grayscale
    map_img[map_np == 100] = 100#Change to [0,0,0] for color to [100] for grayscale
    origin_color = 100#Change to [0,0,255] for color to [0] for grayscale
    robot_color = 100#Change to [0,0,255] for color to [0] for grayscale
# Origin (Drawn as a cross)

    map_img[int(o_y)+0][int(o_x)+0] = origin_color
    map_img[int(o_y)+0][int(o_x)+1] = origin_color
    map_img[int(o_y)+0][int(o_x)+2] = origin_color
    map_img[int(o_y)+1][int(o_x)+0] = origin_color
    map_img[int(o_y)+2][int(o_x)+0] = origin_color
    if (-origin.x) != 0:
        map_img[int(o_y)+0][int(o_x)-1] = origin_color
        map_img[int(o_y)+0][int(o_x)-2] = origin_color
    if (-origin.y) != 0:
        map_img[int(o_y)-1][int(o_x)+0] = origin_color
        map_img[int(o_y)-2][int(o_x)+0] = origin_color
# Robot (Drawn as a square)
    map_img[int(robot_y)+0][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+0][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+0][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)+2][int(robot_x)-1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)-1] = robot_color
    print time.time()-start_time, "background"
    return map_img

In [8]:
def draw_ray(map_img,slices):
    start_time = time.time()
    global display_robot_yaw
    global bearings
    global tags
    global areas
    global robot_x
    global robot_y
    global map_resolution
    global areas
    
    slices_size = len(slices)
    
    if not display_robot_yaw:
        for i in range(int(slices_size)):
            try:
                arr = slices[i]
                appr_dist_low = (1.0-2.5*dist_error)*map_resolution*2*456.7946397925*math.pow(areas[i],-0.4705300238)
                for ray in arr:
                    if ray[3] > appr_dist_low: 
                        if ray[2] == -1 or ray[2] == 0:
                            if False: #display true if you want to see slice
                                map_img[ray[1]][ray[0]] = [255,0,0]#Change to [255,0,0] for color to [0] for grayscale
                        if ray[2] == 100:
                            if tags[i] == 2:
                                map_img[ray[1]][ray[0]]=30#Change to [0,255,0] for color to 0,[180 for grayscale]
                            if tags[i] == 3: 
                                map_img[ray[1]][ray[0]]=50#Change to [0,0,255] for color to 0,[230] for grayscale
            except Exception as ex:
                print ex
    else:
        for arr in slices[-1]:
            map_img[arr[1]][arr[0]] = [0,255,0]
    print time.time()-start_time, "draw ray"
    return map_img

In [9]:
def get_mapnbot_info(trans,robot_yaw):
    start_time = time.time()
    global map_resolution
    global robot_x
    global robot_y
    global map_width
    global map_height
    global origin
    global occ_map
    global map_np
    
    map_array = occ_map.data
    
    map_width = occ_map.info.width
    map_height = occ_map.info.height
    
    map_np = np.reshape(np.array(map_array),(map_height,map_width))

    robot_x = (trans[0]-origin.x)/map_resolution
    robot_y = (trans[1]-origin.y)/map_resolution
    robot_yaw = (90 - robot_yaw)
    while robot_yaw > 360:
        robot_yaw = robot_yaw - 360
    while robot_yaw < 0:
        robot_yaw = robot_yaw + 360
    print time.time()-start_time, "map/bot info"
    return map_np, robot_yaw

In [10]:
def insidetriangle((x1,x2,x3),(y1,y2,y3),map_np):
    start_time = time.time()
    global robot_x
    global robot_y
    
    xs=np.array((x1,x2,x3),dtype=float)
    ys=np.array((y1,y2,y3),dtype=float)

    # The possible range of coordinates that can be returned
    x_range=np.arange(np.min(xs),np.max(xs)+1)
    y_range=np.arange(np.min(ys),np.max(ys)+1)

    # Set the grid of coordinates on which the triangle lies. The centre of the
    # triangle serves as a criterion for what is inside or outside the triangle.
    X,Y=np.meshgrid( x_range,y_range )
    xc=np.mean(xs)
    yc=np.mean(ys)

    # From the array 'triangle', points that lie outside the triangle will be
    # set to 'False'.
    triangle = np.ones(X.shape,dtype=bool)
    for i in range(3):
        ii=(i+1)%3
        if xs[i]==xs[ii]:
            include = X *(xc-xs[i])/abs(xc-xs[i]) > xs[i] *(xc-xs[i])/abs(xc-xs[i])
        else:
            poly=np.poly1d([(ys[ii]-ys[i])/(xs[ii]-xs[i]),ys[i]-xs[i]*(ys[ii]-ys[i])/(xs[ii]-xs[i])])
            include = Y *(yc-poly(xc))/abs(yc-poly(xc)) > poly(X) *(yc-poly(xc))/abs(yc-poly(xc))
        triangle*=include

    # Output: 2 arrays with the x- and y- coordinates of the points inside the
    # triangle.
    

    x_points = X[triangle]
    y_points = Y[triangle]

    points = np.column_stack((x_points.astype(int), y_points.astype(int)))

    itbuffer = np.empty(shape=((points.size/2),4), dtype=np.int32)
    itbuffer.fill(np.nan)

    itbuffer[:,0] = x_points
    itbuffer[:,1] = y_points
    itbuffer[:,2] = map_np[points[:,1],points[:,0]]

# Get distance values    

    deltax = itbuffer[:,0]-robot_x
    deltay = itbuffer[:,1]-robot_y
    
    deltax = map_resolution*deltax
    deltay = map_resolution*deltay
    
    
    deltax = np.square(deltax)
    deltay = np.square(deltay)

    
    dist_sq = deltax+deltay
    
    dist_sq = np.sqrt(dist_sq)
#     dist_sq = map_resolution*dist_sq
    
    itbuffer[:,3] = dist_sq
#     print robot_x,robot_y
#     print np.column_stack((deltax.astype(int),deltay.astype(int)))
    print time.time()-start_time, "inside_triangle"
    return itbuffer

In [11]:
def get_points(buoy_yaw, index):
    start_time = time.time()
    global robot_x
    global robot_y
    global camera_range
    global tri_width
    global areas
    global dist_error
    
    appr_dist_up = (1.0+2.0*dist_error)*2*456.7946397925*math.pow(areas[index],-0.4705300238)
    point_deltax = appr_dist_up * math.sin(math.radians(buoy_yaw)) # appr_dist_up
    point_deltay = appr_dist_up * math.cos(math.radians(buoy_yaw)) #appr_dist_up
    per_slope = -1.0/(point_deltay/point_deltax)
    point1 = np.array([round(robot_x), round(robot_y)], dtype=int)
    point2 = np.array([round(robot_x+point_deltax), round(robot_y+point_deltay)], dtype=int)

    
    
    point3_x = (robot_x+point_deltax) + math.sqrt(math.pow(tri_width,2)/(math.pow(per_slope,2)+1))
    point4_x = (robot_x+point_deltax) - math.sqrt(math.pow(tri_width,2)/(math.pow(per_slope,2)+1))

    point3_y = robot_y+point_deltay + per_slope*(point3_x-(robot_x+point_deltax))
    point4_y = robot_y+point_deltay + per_slope*(point4_x-(robot_x+point_deltax))

    point3 = np.array([round(point3_x),round(point3_y)], dtype=int)
    point4 = np.array([round(point4_x),round(point4_y)], dtype=int)
    
    print time.time()-start_time, "get_points"
    return point1,point2,point3,point4

In [12]:
def map_filter(trans, robot_yaw):
    start_time = time.time()
    global origin
    global robot_x
    global robot_y
    global camera_range
    global occ_map
    global bearings
    global display_robot_yaw
    global tags
    
    map_np, robot_yaw = get_mapnbot_info(trans,robot_yaw) 
    
    slices = []
    i = 0
    while i < len(bearings):
        
        
        buoy_yaw = robot_yaw + bearings[i]
        if buoy_yaw < 0:
            buoy_yaw = buoy_yaw + 360
#         print bearings
##############################################################################
# This is for visualization purposes.
# Set to false if you don't want to display the robot_yaw
# Delete it if it's irritating to look at
        if display_robot_yaw and occ_map.header.seq % 5 == 0:
            bearings.append(0.0)
            print "Robot_yaw: ", robot_yaw
            print "Buoy yaw: ", buoy_yaw
            print len(slices)
##############################################################################
        
        pt1, pt2, pt3, pt4 = get_points(buoy_yaw, i)    
        
#         sliced = createLineIterator(pt1.astype(int),pt2.astype(int),map_np)

        sliced = insidetriangle((pt3[0],pt1[0],pt4[0]),(pt3[1],pt1[1],pt4[1]),map_np)
        slices.append(np.array(sliced).astype(int))
#         print slices
        i+=1
    print time.time()-start_time, "map_filter"
    return slices

In [13]:
def get_map(slices):
    start_time = time.time()
    global camera_range
    global origin
    global robot_x
    global robot_y
    global map_width
    global map_height
    global occ_map
    
    map_np = np.reshape(np.array(occ_map.data),(map_height,map_width))

    #Set to 1 when publishing, to 3 for visualization
    
    map_img_z = np.zeros((map_height, map_width))
    
    map_img_b = draw_background_objects(map_img_z,map_np)
    
    map_img_r = draw_ray(map_img_b,slices)
    print time.time()-start_time, "get_map"
    return map_img_r

In [14]:
def callback(rect):
    start_time = time.time()
    global occ_map
    global origin
    global bearings
    global listener 
    global display_rect
    global display_map
    global tags
    global up_occ_map
    global pub
    global map_height
    global map_width
    global b_list
    
    origin = occ_map.info.origin.position
    rect = transform(rect)
    post = time.time()
    if not rospy.is_shutdown():       
        now = rospy.Time.now()
        listener.waitForTransform("/odom", "/front_left_camera_link", now, rospy.Duration(1))
        (trans,rot) = listener.lookupTransform("/odom", "/front_left_camera_link", now)
        euler = tf.transformations.euler_from_quaternion(rot)
        pixel_lines = map_filter(trans, math.degrees(euler[2]))
        map_image = get_map(pixel_lines)
        map_obj = EnvMap.EnvMap(map_image,[int(origin.y),int(origin.x)])
        buoy_data = map_obj.get_buoys()
        for item in buoy_data:
            b_list.add_buoy(item)
        map_obj.set_buoys(b_list.get_confirmed_buoys())
        map_image = map_obj.get_map()
        if True: #set to false if you don't want to publish the updated occupancy map
            up_occ_map.data = np.array(map_image).flatten()
            publish = time.time()
            if pub.get_num_connections > 0:
                pub.publish(up_occ_map)
                print time.time()-publish, "publish"
        if display_rect:
            disp_rect(rect)
        if display_map:
            disp_map(map_image)
    print time.time()-post, "post"
    print time.time()-start_time, "callback"
    for i in range(5):
        print""

In [15]:
def map_callback(p_map):
    
    global occ_map
    global up_occ_map
    
    occ_map = p_map    
    up_occ_map = p_map

In [16]:
if __name__ == '__main__':
    global b_list
    b_list = Buoy_List.BuoyList()
    proj_map = rospy.Subscriber("projected_map", Map, map_callback)
    rect = rospy.Subscriber("stereo/left/image_rect_color", Image, callback)
    rospy.spin()   

0.0835959911346 preprocessing
0.0118401050568 color hsv
0.00410580635071 color hsv
0.0100419521332 color hsv
0.00796413421631 color hsv
0.00275993347168 color hsv
0.0030369758606 color hsv
0.0030689239502 color hsv
0.00398707389832 color hsv
0.00372791290283 color hsv
0.00301694869995 color hsv
0.0031909942627 color hsv
0.00262308120728 color hsv
0.0021641254425 color hsv
0.00282192230225 color hsv
0.00683498382568 color hsv
0.00286388397217 color hsv
0.00227999687195 color hsv
0.00489902496338 color hsv
0.00353908538818 color hsv
0.00236296653748 color hsv
0.0034511089325 color hsv
0.0034339427948 color hsv
0.00214791297913 color hsv
0.00350403785706 color hsv
0.00270104408264 color hsv
0.00289011001587 color hsv
0.00392007827759 color hsv
0.00477004051208 color hsv
0.00210499763489 color hsv
0.00342321395874 color hsv
0.00366401672363 color hsv
0.00421094894409 color hsv
0.0058741569519 color hsv
0.00438809394836 color hsv
0.00273299217224 color hsv
0.00275421142578 color hsv
0.00407